In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import loss_landscapes
import loss_landscapes.metrics

import utils
import numpy as np

In [14]:
# training hyperparameters
BATCH_SIZE = 64
TEST_BATCH_SIZE = 1000
EPOCHS = 10
LR = 10 ** -2
GAMMA = 0.7 #learning rate step gamma
LOG_INTERVAL = 1000

# contour plot resolution
STEPS = 40

use_cuda = True
if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, criterion, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [16]:
train_kwargs = {'batch_size': BATCH_SIZE}
test_kwargs = {'batch_size': TEST_BATCH_SIZE}

In [17]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('./data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [18]:
device = torch.device("cuda")

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=LR)
criterion = torch.nn.CrossEntropyLoss()

scheduler = StepLR(optimizer, step_size=1, gamma=GAMMA)
for epoch in range(1, EPOCHS + 1):
    train(model, criterion, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.321792

Test set: Average loss: 0.2928, Accuracy: 9129/10000 (91%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.369205

Test set: Average loss: 0.2317, Accuracy: 9308/10000 (93%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.239937

Test set: Average loss: 0.2023, Accuracy: 9389/10000 (94%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.210574

Test set: Average loss: 0.1851, Accuracy: 9447/10000 (94%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.265958

Test set: Average loss: 0.1758, Accuracy: 9476/10000 (95%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.214160

Test set: Average loss: 0.1686, Accuracy: 9493/10000 (95%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.232308

Test set: Average loss: 0.1646, Accuracy: 9511/10000 (95%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.229902

Test set: Average loss: 0.1622, Accuracy: 9518/10000 (95%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.178783

Test set: Average loss: 0.1601, Accuracy: 9521/10000 (95%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.194139


In [19]:
# Get all the model parameters
params = [p for p in model.parameters()]

# Calculate the standard deviation of all parameters
wt_std = torch.std(torch.cat([p.view(-1) for p in params]))

DISTANCE=10*wt_std.cpu().detach().numpy()

print(DISTANCE)

0.08689654991030693


In [20]:
perturbation_range = np.round(np.linspace(-0.5*DISTANCE, 0.5*DISTANCE, 8), 3)

In [ ]:
x, y = iter(test_loader).__next__()

metric = loss_landscapes.metrics.Loss(criterion, x, y)

loss_data_fin = loss_landscapes.random_plane(model.cpu(), metric, distance=DISTANCE, steps=STEPS, normalization='filter', deepcopy_model=True)

utils.plot_loss_contours(loss_data=loss_data_fin, steps=STEPS, distance=DISTANCE, save_fig=True)